In [1]:
import cv2
from ultralytics import YOLO
import numpy as np

# Load YOLO model
model = YOLO("best.pt")

# Capture video from file
cap = cv2.VideoCapture("C:/Users/chait/OneDrive/Desktop/SmartTrafficLight/stock-footage-new-delhi-india-dec-centralized-accidents-and-trauma-services-cats-advanced-life.webm")
assert cap.isOpened(), "Error reading video file"

# Get video properties
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define region points as a polygon (as tuples of (x, y) coordinates)
# region_points = [(170, 90), (4, 276), (586, 272), (429, 88)]

# Convert region points to NumPy array for polygon operations
# region_polygon = np.array(region_points, np.int32).reshape((-1, 1, 2))

# Video writer setup
video_writer = cv2.VideoWriter("object_counting_output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Define total number of vehicles at the intersection and cycle time (for green signal time calculation)
total_vehicles_at_intersection = 100  # Example value, you can change it
cycle_time = 60  # Example cycle time in seconds

# Define class names and corresponding box colors
class_names = {
    0: "auto-rikshaw",
    1: "bike",
    2: "bus",
    3: "car",
    4: "priority",  # Priority vehicles (e.g., ambulance, fire truck)
    5: "truck"
}
box_colors = {
    "auto-rikshaw": (255, 0, 0),  # Blue
    "bike": (0, 255, 0),          # Green
    "bus": (0, 0, 255),           # Red
    "car": (255, 255, 0),         # Cyan
    "priority": (0, 255, 255),    # Yellow
    "truck": (255, 0, 255)        # Magenta
}

priority_detected = False  # Flag to track priority vehicles

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Perform detection with YOLO model
    results = model(frame)

    # Count vehicles inside the polygon
    vehicles_in_region = 0

    for result in results:
        for i, box in enumerate(result.boxes.xyxy):
            # Get class label and confidence
            class_id = int(result.boxes.cls[i])
            class_name = class_names.get(class_id, "unknown")

            # Calculate the center of the bounding box (midpoint of vehicle)
            bbox_center = (int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2))

            # Check if the vehicle's center point is inside the polygon
            # if cv2.pointPolygonTest(region_polygon, bbox_center, False) >= 0:
            #     vehicles_in_region += 1

            #     # Check for priority vehicles
            if class_name == "priority":
                priority_detected = True

            # Draw bounding box for the detected vehicle
            color = box_colors.get(class_name, (255, 255, 255))  # Default to white if class not recognized
            cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), color, 2)

            # Label the box with the class name
            cv2.putText(frame, class_name, (int(box[0]), int(box[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Calculate the green signal time for the lane
    if priority_detected:
        green_signal_time = "Extended"
    elif vehicles_in_region > 0:
        green_signal_time = round((vehicles_in_region / total_vehicles_at_intersection) * cycle_time, 2)
    else:
        green_signal_time = 0

    # Display the number of vehicles inside the polygon region and green signal time on the frame
    cv2.putText(frame, f"Vehicles: {vehicles_in_region}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Green Signal Time: {green_signal_time} sec", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Draw the polygon region
    # cv2.polylines(frame, [region_polygon], isClosed=True, color=(255, 0, 0), thickness=2)

    # Write the frame to the output video file
    video_writer.write(frame)

    # Display the frame using cv2.imshow
    cv2.imshow("Traffic Video", frame)

    # Break on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
video_writer.release()
cv2.destroyAllWindows()



0: 384x640 2 buss, 1 car, 153.6ms
Speed: 19.0ms preprocess, 153.6ms inference, 84.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 1 car, 43.2ms
Speed: 8.0ms preprocess, 43.2ms inference, 14.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 1 car, 84.5ms
Speed: 5.0ms preprocess, 84.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 1 car, 81.7ms
Speed: 8.0ms preprocess, 81.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 1 car, 58.0ms
Speed: 6.0ms preprocess, 58.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 1 car, 89.5ms
Speed: 5.3ms preprocess, 89.5ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 1 car, 51.6ms
Speed: 5.0ms preprocess, 51.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 1 car, 49.3ms
Speed: 7.0ms preprocess, 49.3ms inference, 1.0ms